In [3]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

# 构造数据
def synthetic_data(w, b, number_examples):
    X = torch.normal(0, 1, (number_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape(-1, 1)

# 打乱数据并取小批量 
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i:min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

# 线性回归
def linreg(X, w, b): 
    return torch.matmul(X, w) + b

# 平方损失
def squared_loss(y_hat, y): 
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

# 小批量随机梯度下降
def sgd(params, lr, batch_size): 
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

# 真实参数
true_w = torch.tensor([2, -3.4]) 
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)
batch_size = 10

# 模型参数
w = torch.zeros(2,1, requires_grad = True)
b = torch.zeros(1, requires_grad = True)

lr = 0.03 # 学习率
num_epochs = 3 # 训练轮数
net = linreg # 采用的网络
loss = squared_loss # 采用的损失函数

# 训练
for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)
        l.sum().backward()
        sgd([w, b], lr, batch_size)
        
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

print("w的误差:", true_w - w.reshape(true_w.shape))
print("b的误差:", true_b - b)

epoch 1, loss 0.034166
epoch 2, loss 0.000111
epoch 3, loss 0.000046
w的误差: tensor([ 0.0001, -0.0005], grad_fn=<SubBackward0>)
b的误差: tensor([0.0005], grad_fn=<RsubBackward1>)
